### **Importing necessary libraries**

In [1]:
import pandas as pd
import requests
import time
import traceback

In [2]:
x = pd.read_csv("billboard 2020-2025.csv")


#### **finding the percentage of null values**

In [ ]:
x.isna().mean()*100

**filling the null values in genres column from another website** `musicbrainz`

In [ ]:
headers = {
    "user-agent": "Zeina V1.0.0",
}

URL_BASE = "https://musicbrainz.org/ws/2/"


empty_genres = x[x["genre(s)"].isna()]

genres_cache = {}   #to save up on processing time

count = 0

In [ ]:
for idx, row in empty_genres.iterrows():
    try:
        if count % 25 == 0:
            print(f"At row {int(idx) + 1} out of {len(x)}.")
            x.to_csv("billboard_complete.csv", index=False)

        count += 1

        # Search for artist
        # https://musicbrainz.org/ws/2/artist/?query=Mariah%20Carey&fmt=json&limit=1&inc=genres
        artist_name = row["performer"]

        if artist_name in genres_cache:
            x.loc[idx, "genre(s)"] = genres_cache[artist_name]
            continue

        resp = requests.get(f"{URL_BASE}artist/?query={artist_name}&fmt=json&limit=1")
        if not resp.ok:
            print(f"Error: {resp.status_code}")
            break

        body = resp.json()
        artist_mbid = body["artists"][0]["id"]

        time.sleep(1)

        # https://musicbrainz.org/ws/2/artist/5b11f4ce-a62d-471e-81fc-a69a8278c7da?inc=genres&fmt=json

        url = f"{URL_BASE}artist/{artist_mbid}?inc=genres&fmt=json"
        print(url)
        resp = requests.get(f"{URL_BASE}artist/{artist_mbid}?inc=genres&fmt=json")
        if not resp.ok:
            print(f"Error: {resp.status_code}")
            break

        body = resp.json()
        artist_genres = body["genres"]
        flattened_genres = []
        for genre in artist_genres:
            flattened_genres.append(genre["name"])

        x.loc[idx, "genre(s)"] = ",".join(flattened_genres)
        genres_cache[artist_name] = ",".join(flattened_genres)
        print(f"Set Genres for artist {artist_name} at row {idx}")

        time.sleep(1)
    except Exception as e:
        print(traceback.format_exc())
        print("Error saving gracefully")
        x.to_csv("billboard_complete.csv", index=False)

x.to_csv("billboard_complete.csv", index=False)


### **operating on the completed file**

In [52]:
x=pd.read_csv("billboard_complete.csv")

In [53]:
x.isna().mean()*100

chart_week      0.000000
current_week    0.000000
title           0.000000
performer       0.000000
last_week       0.000000
peak_pos        0.000000
wks_on_chart    0.000000
genre(s)        3.469314
explicit        0.000000
duration-ms     0.000000
tempo           0.003610
loudness        0.003610
energy          0.003610
danceability    0.003610
dtype: float64

In [40]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27700 entries, 0 to 27699
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   chart_week    27700 non-null  object 
 1   current_week  27700 non-null  int64  
 2   title         27700 non-null  object 
 3   performer     27700 non-null  object 
 4   last_week     25884 non-null  object 
 5   peak_pos      27700 non-null  int64  
 6   wks_on_chart  27700 non-null  int64  
 7   genre(s)      26739 non-null  object 
 8   explicit      27700 non-null  bool   
 9   duration-ms   27700 non-null  float64
 10  tempo         27699 non-null  float64
 11  loudness      27699 non-null  float64
 12  energy        27699 non-null  float64
 13  danceability  27699 non-null  float64
 14  old_genre(s)  27700 non-null  bool   
dtypes: bool(2), float64(5), int64(3), object(5)
memory usage: 2.8+ MB


### **changing data types of columns to the appropriate data type**

In [54]:
x["last_week"].unique()

array([  1,   2,   9,   6,   3,   4,  15,   8,   7,  10,  17,  23,  18,
        11,  28,  30,  13,  12,  14,  16,  37,  36,  25,  40,  20,  21,
        19,  50,   0,  29,  47,  31,  41,  32,  27,  26,  34,  53,  33,
        35,   5,  39,  38,  42,  24,  44,  51,  58,  48,  46,  49,  43,
        52,  61,  57,  55,  56,  59,  65,  60,  70,  68,  63,  66,  71,
        72,  69,  76,  77,  75,  81,  78,  73,  74,  91,  89,  54,  96,
        88,  97,  85,  93,  82,  95,  80,  94,  92,  98,  64,  62,  67,
        86, 100,  84,  90,  87,  99,  83,  22,  45,  79], dtype=int64)

In [55]:
x['last_week'].replace('-' , '0').astype(object)

0        1
1        2
2        9
3        6
4        3
        ..
27695    0
27696    0
27697    0
27698    0
27699    0
Name: last_week, Length: 27700, dtype: object

In [43]:
x["chart_week"] =pd.to_datetime(x["chart_week"])
x["last_week"] =x['last_week'].replace('-' , '0').astype(object)
x["last_week"] = pd.to_numeric(x["last_week"])

x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27700 entries, 0 to 27699
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   chart_week    27700 non-null  datetime64[ns]
 1   current_week  27700 non-null  int64         
 2   title         27700 non-null  object        
 3   performer     27700 non-null  object        
 4   last_week     25884 non-null  float64       
 5   peak_pos      27700 non-null  int64         
 6   wks_on_chart  27700 non-null  int64         
 7   genre(s)      26739 non-null  object        
 8   explicit      27700 non-null  bool          
 9   duration-ms   27700 non-null  float64       
 10  tempo         27699 non-null  float64       
 11  loudness      27699 non-null  float64       
 12  energy        27699 non-null  float64       
 13  danceability  27699 non-null  float64       
 14  old_genre(s)  27700 non-null  bool          
dtypes: bool(2), datetime64[ns](1), float

In [56]:
x["last_week"] =x['last_week'].fillna(0)
x["last_week"] = x['last_week'].apply(int)

In [57]:
del x['old_genre(s)']

KeyError: 'old_genre(s)'

In [58]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27700 entries, 0 to 27699
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   chart_week    27700 non-null  object 
 1   current_week  27700 non-null  int64  
 2   title         27700 non-null  object 
 3   performer     27700 non-null  object 
 4   last_week     27700 non-null  int64  
 5   peak_pos      27700 non-null  int64  
 6   wks_on_chart  27700 non-null  int64  
 7   genre(s)      26739 non-null  object 
 8   explicit      27700 non-null  bool   
 9   duration-ms   27700 non-null  float64
 10  tempo         27699 non-null  float64
 11  loudness      27699 non-null  float64
 12  energy        27699 non-null  float64
 13  danceability  27699 non-null  float64
dtypes: bool(1), float64(5), int64(4), object(4)
memory usage: 2.8+ MB


### **saving the changes**

In [59]:
x.to_csv("billboard_complete.csv", index=False)

In [48]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27700 entries, 0 to 27699
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   chart_week    27700 non-null  datetime64[ns]
 1   current_week  27700 non-null  int64         
 2   title         27700 non-null  object        
 3   performer     27700 non-null  object        
 4   last_week     27700 non-null  int64         
 5   peak_pos      27700 non-null  int64         
 6   wks_on_chart  27700 non-null  int64         
 7   genre(s)      26739 non-null  object        
 8   explicit      27700 non-null  bool          
 9   duration-ms   27700 non-null  float64       
 10  tempo         27699 non-null  float64       
 11  loudness      27699 non-null  float64       
 12  energy        27699 non-null  float64       
 13  danceability  27699 non-null  float64       
 14  old_genre(s)  27700 non-null  bool          
dtypes: bool(2), datetime64[ns](1), float

In [60]:
df

,chart_week,current_week,title,performer,last_week,peak_pos,wks_on_chart,genre(s),explicit,duration-ms,tempo,loudness,energy,danceability
0,2020-01-04,1,All I Want For Christmas Is You,Mariah Carey,1,1,37,christmas,False,241106.0,150.273,-7.463,0.627,0.336
1,2020-01-04,2,Rockin' Around The Christmas Tree,Brenda Lee,2,2,32,christmas,False,126266.0,67.086,-8.744,0.470,0.598
2,2020-01-04,3,Jingle Bell Rock,Bobby Helms,9,3,30,christmas,False,130973.0,119.705,-8.463,0.424,0.754
3,2020-01-04,4,A Holly Jolly Christmas,Burl Ives,6,4,15,christmas,False,135533.0,140.453,-13.056,0.375,0.682
4,2020-01-04,5,Circles,Post Malone,3,1,17,"contemporary r&b,country,hip hop,pop rap,trap",False,215280.0,120.042,-3.497,0.763,0.695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27695,2025-01-11,96,Holy Smokes,Bailey Zimmerman,0,54,7,country,False,195157.0,95.117,-7.908,0.569,0.462
27696,2025-01-11,97,High Road,Zach Bryan,0,29,5,"country,red dirt",True,199893.0,113.404,-11.028,0.229,0.470
27697,2025-01-11,98,On One Tonight,Gunna,0,54,7,melodic rap,True,90760.0,130.035,-4.318,0.727,0.887
27698,2025-01-11,99,St. Chroma,"Tyler, The Creator Featuring Daniel Caesar",0,7,9,"egyptian hip hop,arabic rap,egyptian pop",False,195297.0,101.096,-7.774,0.724,0.671
